# Personalize 순위 예 <a class="anchor" id="top"></a>

이 노트북에서는 데이터 세트를 선택하고 Amazon Personalize 배치 추천에 사용할 수 있도록 준비합니다.

1. [데이터 세트 또는 데이터 소스 선택](#source)
1. [데이터 준비](#prepare)
1. [데이터 세트 그룹 및 상호 작용 데이터 세트 생성](#group_dataset)
1. [S3 버킷 및 IAM 역할 구성](#bucket_role)
1. [상호 작용 데이터 가져오기](#import)
1. [솔루션 생성](#solutions)
1. [캠페인 생성](#create)
1. [캠페인과 상호 작용](#interact)
1. [정리](#cleanup)

## 소개 <a class="anchor" id="intro"></a>

대부분의 경우, Amazon Personalize(레시피라고 함)의 알고리즘은 여기서 설명하는 다양한 작업을 처리합니다.

1. **HRNN 및 HRNN 메타데이터** - 이전 사용자 상호 작용에 따라 항목을 추천합니다.
1. **HRNN 콜드 스타트** - 아직 상호 작용 데이터가 확보되지 않은 새 항목을 추천합니다.
1. **개인별 순위** - 항목 모음을 선택한 다음 HRNN과 유사한 접근 방식을 사용하여 항목 순서를 지정합니다.
1. **SIMS(유사 항목)** - 특정 항목이 주어지면 사용자가 상호 작용하는 다른 항목도 추천합니다.
1. **인기 카운트** - HRNN 또는 HRNN 메타데이터에서 적절한 정보를 찾을 수 없는 경우 가장 인기 있는 항목을 추천합니다. 이 항목은 기본적으로 반환됩니다.

사용 사례에 관계없이, 모든 알고리즘은 3가지 핵심 속성으로 정의된 사용자-항목-상호 작용 데이터에 대한 학습 기반을 공유합니다.

1. **UserID** - 상호 작용한 사용자
1. **ItemID** - 사용자가 상호 작용한 항목
1. **타임스탬프** - 상호 작용이 발생한 시간

또한 다음과 같이 정의된 이벤트 유형과 이벤트 값도 지원합니다.

1. **이벤트 유형** - 이벤트의 범주형 레이블(찾아보기, 구매, 평점 등)입니다.
1. **이벤트 값** - 발생한 이벤트 유형에 해당하는 값입니다. 간단히 말해, 이벤트 유형에 대해 0과 1 사이의 정규화된 값을 찾는 것입니다. 예를 들어 3단계(클릭, 장바구니에 추가, 구매)로 거래를 완료하는 경우 각 단계마다 event_value는 각각 0.33, 0.66, 1.0이 됩니다.

이벤트 유형 및 이벤트 값 필드는 개인화 모델 훈련을 위해 전송된 데이터를 필터링하는 데 사용할 수 있는 추가 데이터입니다. 이 연습에서는 이벤트 유형이나 이벤트 값이 없습니다.

## 데이터 세트 또는 데이터 소스 선택 <a class="anchor" id="source"></a>
[맨 위로 이동](#top)

앞서 언급했듯이, 사용자-항목-상호 작용 데이터는 서비스를 시작하기 위한 핵심 속성입니다. 즉, 이러한 종류의 데이터를 생성하는 사용 사례를 찾아야 합니다. 몇 가지 일반적인 예는 다음과 같습니다.

1. 비디오 온디맨드 애플리케이션
1. 전자 상거래 플랫폼
1. 소셜 미디어 집계기/플랫폼

Personalize에 적합한 문제의 범위를 지정하는 데 적용되는 몇 가지 지침이 있습니다. [공식 한도](https://docs.aws.amazon.com/personalize/latest/dg/limits.html)는 조금 더 낮지만 아래의 값을 시작점으로 사용하면 유용합니다.

* 인증된 사용자
* 50명 이상의 고유한 사용자
* 100개 이상의 고유한 항목
* 사용자당 24건 이상의 상호 작용 

대부분의 경우 이는 쉽게 달성할 수 있으며 한 범주에서 값이 기준에 못 미치면 다른 범주에서 더 많은 수를 확보하여 보충할 수 있습니다.

일반적으로 데이터는 Personalize에 사용하기에 완벽한 형식으로 제공되지 않으며, 올바르게 구성되기 위해서는 약간의 수정이 필요합니다. 이 노트북에서는 그러한 모든 과정을 안내합니다.

먼저, [Last.FM](https://grouplens.org/datasets/hetrec-2011/) 데이터 세트를 사용해 보겠습니다. 이 데이터 세트는 사용자들의 음악 감상 행동에 대한 기록입니다. 이 데이터는 다수의 사용자, 항목 및 상호 작용에 대한 우리의 지침에 부합합니다.

먼저 아래의 코드를 사용하여 데이터 세트를 다운로드하고 새 폴더에 압축을 풉니다.

In [4]:
data_dir = "data"
!mkdir $data_dir
!cd $data_dir && wget http://files.grouplens.org/datasets/hetrec2011/hetrec2011-lastfm-2k.zip
!cd $data_dir && unzip hetrec2011-lastfm-2k.zip

--2020-05-26 20:45:51--  http://files.grouplens.org/datasets/hetrec2011/hetrec2011-lastfm-2k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2589075 (2.5M) [application/zip]
Saving to: ‘hetrec2011-lastfm-2k.zip’

hetrec2011-lastfm-2 100%[===================>]   2.47M  10.2MB/s    in 0.2s    

2020-05-26 20:45:51 (10.2 MB/s) - ‘hetrec2011-lastfm-2k.zip’ saved [2589075/2589075]

Archive:  hetrec2011-lastfm-2k.zip
  inflating: user_friends.dat        
  inflating: user_taggedartists.dat  
  inflating: user_taggedartists-timestamps.dat  
  inflating: artists.dat             
  inflating: readme.txt              
  inflating: tags.dat                
  inflating: user_artists.dat        


다운로드한 데이터 파일을 살펴봅니다.

In [5]:
!ls $data_dir

artists.dat		  tags.dat	    user_taggedartists.dat
hetrec2011-lastfm-2k.zip  user_artists.dat  user_taggedartists-timestamps.dat
readme.txt		  user_friends.dat


현재 많은 .dat 파일과 README가 있다는 것 외에는 데이터에 대해 알려진 사실이 거의 없습니다. README를 열어보면 이 데이터의 전체 구조에 대해 알 수 있습니다. 데이터 소스가 외부 팀에서 제공되지 않는 한 사용자 지정 데이터를 사용하고 이 단계를 건너뛸 수 있습니다.

README에서, 이 데이터 세트에는 여러 가지 상호 작용 유형이 있음을 확인할 수 있습니다. 서로를 친구로 지정한 사용자 간의 상호 작용, 아티스트의 음악을 감상하는 사용자의 상호 작용, 사용자와 아티스트에 할당된 태그를 사용한 상호 작용 등입니다.

이 에에서는 사용자, 아티스트 및 음악 감상 상호 작용에 초점을 맞춥니다. 1,892명의 사용자, 17,632명의 아티스트(이 예에서 항목에 해당), 사용자가 아티스트의 음악을 감상하는 상호 작용 92,834건이 있습니다. 이 정도면 Personalize를 시작하는 데 충분합니다.

`Files` 섹션까지 README를 계속 읽습니다. 데이터 세트에 있는 대부분의 파일은 우리와 관련이 없지만, `users_artists.dat` 파일은 유용해 보입니다. README의 `Data format` 섹션에는 이 파일의 콘텐츠에 대한 세부 정보가 나와 있습니다. 여기서 첫 번째 문제에 부딪히게 됩니다.

| userID | artistID | weight  |
|--------|----------|---------|
| 2      | 51       | 13883   |

사용자와 사용자가 음악을 감상하는 아티스트 간의 상호 작용 데이터가 있지만, 상호 작용이 타임스탬프가 아니라 가중치로 저장되어 있습니다. Amazon Personalize에는 사용자-항목-타임스탬프 상호 작용 데이터가 필요합니다.

데이터 세트의 파일을 다시 살펴보면 `users_taggedartists-timestamps.dat`에 타임스탬프 데이터가 포함되어 있음을 알 수 있습니다. 음악 감상 행동 대신에 태깅 행동을 상호 작용 데이터로 사용한다면 어떨까요? 사용자가 아티스트를 태깅하는 행동을 긍정적인 감정의 표현으로 간주할 수 있을까요? 일반적으로, 해결하고자 하는 사용 사례에 이 상호 작용이 적합한지 여부를 이해하기 위해서는 고객 또는 도메인에 대해 잘 아는 사람과 논의해야 합니다. 여기서는 태깅 행동이 우리의 요구 사항에 적합하다고 가정하겠습니다.

`user_taggedartists-timestamps.dat`의 스키마는 다음과 같습니다.

| userID | artistID | tagID | timestamp     |
|--------|----------|-------|---------------|
| 2      | 52       | 13    | 1238536800000 |

`tagID` 속성을 제거하면 Amazon Personalize에 필요한 형식을 정확하게 얻을 수 있습니다.

## 데이터 준비 <a class="anchor" id="prepare"></a>
[맨 위로 이동](#top)

다음으로 데이터를 로드하고 데이터가 양호한 상태인지 확인한 다음, 데이터를 CSV에 저장하여 Amazon Personalize에 사용할 준비가 되도록 합니다.

시작하기 위해 데이터 과학에서 일반적으로 사용되는 Python 라이브러리 컬렉션을 가져옵니다.

In [6]:
import time
from time import sleep
import json
from datetime import datetime
import numpy as np
import boto3
import pandas as pd

그런 다음 데이터 파일을 열고 처음 몇 행을 살펴봅니다.

In [7]:
original_data = pd.read_csv(data_dir + '/user_taggedartists-timestamps.dat')
original_data.head(5)

,userID artistID tagID timestamp
0,2\t52\t13\t1238536800000
1,2\t52\t15\t1238536800000
2,2\t52\t18\t1238536800000
3,2\t52\t21\t1238536800000
4,2\t52\t41\t1238536800000


보시다시피, 데이터가 올바르게 로드되지 않았습니다. CSV(쉼표로 구분된 값) 파일의 기본 구분 기호는 쉼표(`,`)이지만 이 예에서는 파일이 탭(`\t`) 구분 기호를 사용하여 저장되었습니다. 따라서 올바른 구분 기호를 지정하고 데이터를 다시 로드해 보겠습니다.

In [8]:
original_data = pd.read_csv(data_dir + '/user_taggedartists-timestamps.dat', delimiter='\t')
original_data.head(5)

,userID,artistID,tagID,timestamp
0,2,52,13,1238536800000
1,2,52,15,1238536800000
2,2,52,18,1238536800000
3,2,52,21,1238536800000
4,2,52,41,1238536800000


이제 제대로 됐습니다. 이제 데이터가 메모리에 성공적으로 로드되었으므로 몇 가지 추가 정보를 추출하겠습니다. 먼저, 데이터에서 몇 가지 기본 통계를 계산합니다.

In [9]:
original_data.describe()

,userID,artistID,tagID,timestamp
count,186479.000000,186479.000000,186479.000000,1.864790e+05
mean,1035.600137,4375.845328,1439.582913,1.239204e+12
std,622.461272,4897.789595,2775.340279,4.299091e+10
min,2.000000,1.000000,1.000000,-4.287204e+11
25%,488.000000,686.000000,79.000000,1.209593e+12
50%,1021.000000,2203.000000,195.000000,1.243807e+12
75%,1624.000000,6714.000000,887.000000,1.275343e+12
max,2100.000000,18744.000000,12647.000000,1.304941e+12


이는 `userID` 및 `artistID` 값의 범위가 충분하다는 것을 보여줍니다. 다음으로, 항상 데이터 형식을 확인하는 것이 좋습니다.

In [10]:
original_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186479 entries, 0 to 186478
Data columns (total 4 columns):
userID       186479 non-null int64
artistID     186479 non-null int64
tagID        186479 non-null int64
timestamp    186479 non-null int64
dtypes: int64(4)
memory usage: 5.7 MB


이와 관련해, 데이터 세트에 4개의 열에 걸쳐 총 186,479개의 항목이 있으며 각 셀은 int64 형식으로 저장되다는 것을 알 수 있습니다.

int64 형식은 확실히 `userID`와 `artistID`에 적합합니다. 하지만 좀 더 자세히 들어가서 데이터의 타임스탬프를 이해할 필요가 있습니다. Amazon Personalize를 사용하려면 타임스탬프를 [Unix Epoch](https://en.wikipedia.org/wiki/Unix_time) 형식으로 저장해야 합니다.

현재 이 타임스탬프 값은 사람이 읽을 수 없습니다. 그럼 임의의 타임스탬프 값을 가지고 해석하는 방법을 알아보겠습니다.

In [11]:
arb_time_stamp = original_data.iloc[50]['timestamp']
print(arb_time_stamp)
print(datetime.utcfromtimestamp(arb_time_stamp).strftime('%Y-%m-%d %H:%M:%S'))

1235862000000


ValueError: year 41132 is out of range

이런! 이 타임스탬프 값의 경우 코드는 41,132년이라는 연도를 렌더링했습니다. 이는 너무 먼 미래이므로, 데이터를 해석하는 방법이 잘못되었다는 것을 확실히 알 수 있습니다. 다시 시도해 보아야 합니다.

JavaScript는 시간을 밀리초 단위로 기록하며, 이 데이터 세트는 웹 애플리케이션에서 데이터를 수집한 것이므로 코드를 적용하기 전에 타임스탬프 값을 1000으로 나누겠습니다.

In [12]:
arb_time_stamp = arb_time_stamp/1000
print(datetime.utcfromtimestamp(arb_time_stamp).strftime('%Y-%m-%d %H:%M:%S'))

2009-02-28 23:00:00


2009년 2월이라는 값은 이 데이터 세트에 대해 훨씬 더 현실적으로 느껴집니다. Amazon Personalize를 사용하는 데 사람이 읽을 수 있는 타임스탬프는 필요 없지만, 날짜를 사실적으로 표시해야 하므로 이제 데이터 세트의 각 타임스탬프를 JavaScript 밀리초 형식 이외의 형식으로 변환합니다. 

In [13]:
original_data.timestamp = original_data.timestamp / 1000
original_data.head(5)

,userID,artistID,tagID,timestamp
0,2,52,13,1.238537e+09
1,2,52,15,1.238537e+09
2,2,52,18,1.238537e+09
3,2,52,21,1.238537e+09
4,2,52,41,1.238537e+09


임의의 타임스탬프를 선택하고 사람이 읽을 수 있는 형식으로 변환하여 변환된 데이터 세트에 대해 간단한 온전성 검사를 수행합니다.

In [14]:
arb_time_stamp = original_data.iloc[50]['timestamp']
print(arb_time_stamp)
print(datetime.utcfromtimestamp(arb_time_stamp).strftime('%Y-%m-%d %H:%M:%S'))

1235862000.0
2009-02-28 23:00:00


이 날짜는 타임스탬프로 적절하므로, 나머지 데이터의 형식을 계속 변환할 수 있습니다. 필요한 데이터는 사용자-항목-상호 작용 데이터이며, 이 예에서는 `userID`, `artistID` 및 `timestamp`입니다. 이 데이터 세트에는 데이터 세트에서 삭제할 수 있는 추가 열인 `tagID`가 있습니다.

In [15]:
interactions_df = original_data.copy()
interactions_df = interactions_df[['userID', 'artistID', 'timestamp']]
interactions_df.head()

,userID,artistID,timestamp
0,2,52,1.238537e+09
1,2,52,1.238537e+09
2,2,52,1.238537e+09
3,2,52,1.238537e+09
4,2,52,1.238537e+09


데이터를 조작한 후에는 항상 데이터 형식이 변경되었는지 확인합니다.

In [16]:
interactions_df.dtypes

userID         int64
artistID       int64
timestamp    float64
dtype: object

이 예에서는 타임스탬프 열이 int64에서 float64로 변경되었습니다. 이제 형식을 int64로 다시 변경하겠습니다.

In [17]:
interactions_df.astype({'timestamp': 'int64'}).dtypes

userID       int64
artistID     int64
timestamp    int64
dtype: object

 Amazon Personalize에는 사용자, 항목 및 타임스탬프의 기본 열 이름이 있습니다. 이러한 기본 열 이름은 `USER_ID`, `ITEM_ID` 및 `TIMESTAMP`입니다. 따라서 데이터 세트의 마지막 수정 사항은 기존 열 헤더를 기본 헤더로 바꾸는 것입니다.

In [18]:
interactions_df.rename(columns = {'userID':'USER_ID', 'artistID':'ITEM_ID', 
                              'timestamp':'TIMESTAMP'}, inplace = True) 

이제 끝났습니다. 이제 데이터가 사용 가능한 상태로 준비되었으므로, CSV 파일로 저장하기만 하면 됩니다.

In [19]:
interactions_filename = "interactions.csv"
interactions_df.to_csv((data_dir+"/"+interactions_filename), index=False, float_format='%.0f')

## 데이터 세트 그룹 및 상호 작용 데이터 세트 생성 <a class="anchor" id="group_dataset"></a>
[맨 위로 이동](#top)

Amazon Personalize에서 격리 및 추상화의 최상위 수준은 데이터 세트 그룹입니다. 이러한 데이터 세트 그룹 중 하나에 저장된 정보는 다른 데이터 세트 그룹이나 거기에서 생성된 모델에 영향을 주지 않으며 완전히 격리됩니다. 이를 통해 많은 실험을 실행할 수 있으며, 이는 모델을 비공개 상태로 사용자의 데이터에 대해서만 완벽하게 훈련된 상태로 유지하기 위한 방법 중 하나입니다.

이전에 준비된 데이터를 가져오기 전에, 데이터 세트 그룹이 필요하고 해당 데이터 세트 그룹에 상호 작용을 처리하는 데이터 세트를 추가해야 합니다.

데이터 세트 그룹은 다음과 같은 유형의 정보를 저장할 수 있습니다.

* 사용자-항목-상호 작용
* 이벤트 스트림(실시간 상호 작용)
* 사용자 메타데이터
* 항목 메타데이터

상호 작용 데이터의 데이터 세트 그룹과 데이터 세트를 만들기 전에, 사용자 환경이 Amazon Personalize와 정상적으로 통신할 수 있는지 검증해 보겠습니다.

In [20]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

### 데이터 세트 그룹 생성

다음 셀은 `personalize-poc-lastfm`이라는 이름으로 새 데이터 세트 그룹을 만듭니다.

In [21]:
create_dataset_group_response = personalize.create_dataset_group(
    name = "personalize-ranking-dsg"
)

dataset_group_arn = create_dataset_group_response['datasetGroupArn']
print(json.dumps(create_dataset_group_response, indent=2))

{
  "datasetGroupArn": "arn:aws:personalize:us-east-1:144386903708:dataset-group/personalize-ranking-dsg",
  "ResponseMetadata": {
    "RequestId": "386224ac-8410-4f64-ba4a-460ba12607c7",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 26 May 2020 20:46:27 GMT",
      "x-amzn-requestid": "386224ac-8410-4f64-ba4a-460ba12607c7",
      "content-length": "102",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


데이터 세트 그룹을 사용하려면 먼저 그룹이 활성화되어 있어야 합니다. 활성화하는 데에는 1~2분이 걸립니다. 아래의 셀을 실행하고 활성 상태로 표시될 때까지 기다립니다. 이 셀은 최대 3시간 동안 매초마다 데이터 세트 그룹의 상태를 확인합니다.

In [22]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

DatasetGroup: CREATE PENDING
DatasetGroup: ACTIVE


이제 데이터 세트 그룹을 만들었으므로 상호 작용 데이터의 데이터 세트를 만들 수 있습니다.

### 데이터 세트 생성

먼저 업로드하려는 데이터 세트의 유형을 Amazon Personalize에 알려주는 스키마를 정의합니다. 스키마에는 데이터 세트 유형에 따라 몇 가지 예약 키워드와 필수 키워드가 필요합니다. 자세한 내용은 [설명서](https://docs.aws.amazon.com/personalize/latest/dg/how-it-works-dataset-schema.html)를 참조하세요.

여기서는 `USER_ID`, `ITEM_ID` 및 `TIMESTAMP` 필드가 필요한 상호 작용 데이터에 대한 스키마를 만듭니다. 데이터 세트에서 나타나는 순서와 동일한 순서로 스키마에 정의해야 합니다.

In [23]:
interactions_schema = schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        }
    ],
    "version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = "personalize-ranking-interactions",
    schema = json.dumps(interactions_schema)
)

schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

{
  "schemaArn": "arn:aws:personalize:us-east-1:144386903708:schema/personalize-ranking-interactions",
  "ResponseMetadata": {
    "RequestId": "d3fb42c4-9a3c-45a4-a842-adf60ef03906",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 26 May 2020 20:47:58 GMT",
      "x-amzn-requestid": "d3fb42c4-9a3c-45a4-a842-adf60ef03906",
      "content-length": "98",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


생성된 스키마를 사용하여 데이터 세트 그룹 내에 데이터 세트를 만들 수 있습니다. 데이터는 아직 로드하지 않는다는 점을 유의하세요. 데이터는 몇 단계 후에 로드합니다.

In [24]:
dataset_type = "INTERACTIONS"
create_dataset_response = personalize.create_dataset(
    name = "personalize-ranking-ds",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = schema_arn
)

interactions_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

{
  "datasetArn": "arn:aws:personalize:us-east-1:144386903708:dataset/personalize-ranking-dsg/INTERACTIONS",
  "ResponseMetadata": {
    "RequestId": "98a3c225-31aa-49be-936f-f633aa6314d0",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 26 May 2020 20:48:02 GMT",
      "x-amzn-requestid": "98a3c225-31aa-49be-936f-f633aa6314d0",
      "content-length": "104",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


## S3 버킷 및 IAM 역할 구성 <a class="anchor" id="bucket_role"></a>
[맨 위로 이동](#top)

지금까지 이 Jupyter 노트북을 실행하는 인스턴스에 연결된 Amazon EBS 인스턴스에 데이터를 다운로드하고 조작하고 저장했습니다. 하지만 Amazon Personalize에는 데이터의 소스 역할을 하는 S3 버킷과 해당 버킷에 액세스하기 위한 IAM 역할이 필요합니다. 이 버킷과 역할을 설정해보겠습니다.

이 Amazon SageMaker 노트북의 기반이 되는 인스턴스에 저장된 메타데이터를 사용하여 해당 노트북이 운영되고 있는 리전을 확인합니다. Amazon SageMaker 외부에서 Jupyter 노트북을 사용하는 경우 아래의 문자열로 리전을 정의하면 됩니다. Amazon S3 버킷은 지금까지 생성한 Amazon Personalize 리소스와 동일한 리전에 있어야 합니다.

In [25]:
with open('/opt/ml/metadata/resource-metadata.json') as notebook_info:
    data = json.load(notebook_info)
    resource_arn = data['ResourceArn']
    region = resource_arn.split(':')[3]
print(region)

us-east-1


Amazon S3 버킷 이름은 전역적으로 고유합니다. 고유한 버킷 이름을 만들기 위해 아래 코드는 AWS 계정 번호에 `personalizepoc` 문자열을 추가합니다. 그런 다음 이전 셀에서 확인된 리전에 이 이름으로 버킷을 만듭니다.

In [26]:
s3 = boto3.client('s3')
suffix = str(np.random.uniform())[4:9]
bucket_name = "personalize-ranking-demo-"+   suffix        # replace with the name of your S3 bucket
print(bucket_name)
if region != "us-east-1":
    s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={'LocationConstraint': region})
else:
    s3.create_bucket(Bucket=bucket_name)

personalize-ranking-demo-41600


### S3에 데이터 업로드

이제 Amazon S3 버킷이 생성되었으므로 사용자-항목-상호 작용 데이터의 CSV 파일을 업로드합니다. 

In [27]:
interactions_file_path = data_dir + "/" + interactions_filename
boto3.Session().resource('s3').Bucket(bucket_name).Object(interactions_filename).upload_file(interactions_file_path)
interactions_s3DataPath = "s3://"+bucket_name+"/"+interactions_filename

### S3 버킷 정책 설정
Amazon Personalize가 S3 버킷의 콘텐츠를 읽을 수 있어야 합니다. 이를 허용하는 버킷 정책을 추가합니다.

In [28]:
policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:*Object",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket_name),
                "arn:aws:s3:::{}/*".format(bucket_name)
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket_name, Policy=json.dumps(policy))

{'ResponseMetadata': {'RequestId': '61279AD0FB422CD2',
  'HostId': 'pvAc3tsFJjLUU5BTudF9gokRma/yRh+uH3TdrVXk3/xoLlHsOev2XPv8KDc+TusZphyzL/TDUvE=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 'pvAc3tsFJjLUU5BTudF9gokRma/yRh+uH3TdrVXk3/xoLlHsOev2XPv8KDc+TusZphyzL/TDUvE=',
   'x-amz-request-id': '61279AD0FB422CD2',
   'date': 'Tue, 26 May 2020 20:48:18 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

### IAM 역할 생성

Amazon Personalize가 특정 작업을 실행할 수 있는 권한을 가지려면, AWS에서 역할을 수임할 수 있어야 합니다. IAM 역할을 생성하고 필요한 정책을 연결해보겠습니다. 아래의 코드는 과도하게 권한을 부여하는 정책을 연결합니다. 프로덕션 애플리케이션에는 보다 제한적인 정책을 사용하세요.

In [29]:
iam = boto3.client("iam")

role_name = "PersonalizeRoleRanking"
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}

create_role_response = iam.create_role(
    RoleName = role_name,
    AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
)

# AmazonPersonalizeFullAccess provides access to any S3 bucket with a name that includes "personalize" or "Personalize" 
# if you would like to use a bucket with a different name, please consider creating and attaching a new policy
# that provides read access to your bucket or attaching the AmazonS3ReadOnlyAccess policy to the role
policy_arn = "arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess"
iam.attach_role_policy(
    RoleName = role_name,
    PolicyArn = policy_arn
)

# Now add S3 support
iam.attach_role_policy(
    PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess',
    RoleName=role_name
)
time.sleep(60) # wait for a minute to allow IAM role policy attachment to propagate

role_arn = create_role_response["Role"]["Arn"]
print(role_arn)

arn:aws:iam::144386903708:role/PersonalizeRoleRanking


## 상호 작용 데이터 가져오기 <a class="anchor" id="import"></a>
[맨 위로 이동](#top)

앞서 데이터 세트 그룹과 데이터 세트를 만들어 정보를 저장했으므로, 이제 S3 버킷에서 Amazon Personalize 데이터 세트로 데이터를 로드하는 가져오기 작업을 실행합니다. 

In [30]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "personalize-ranking-example",
    datasetArn = interactions_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, interactions_filename)
    },
    roleArn = role_arn
)

dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws:personalize:us-east-1:144386903708:dataset-import-job/personalize-ranking-example",
  "ResponseMetadata": {
    "RequestId": "7eab862d-9724-41dc-87d3-7455b4f16ead",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 26 May 2020 20:51:57 GMT",
      "x-amzn-requestid": "7eab862d-9724-41dc-87d3-7455b4f16ead",
      "content-length": "115",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


데이터 세트를 사용하려면 먼저 가져오기 작업이 활성 상태여야 합니다. 아래의 셀을 실행하고 활성 상태로 표시될 때까지 기다립니다. 이 셀은 최대 3시간 동안 매초마다 가져오기 작업의 상태를 확인합니다.

데이터 세트의 크기에 따라 데이터를 가져오는 데 시간이 걸릴 수 있습니다. 이 워크숍에서는 데이터 가져오기 작업에 약 15분이 소요됩니다.

In [31]:
%%time

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_import_job_arn
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    print("DatasetImportJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

DatasetImportJob: CREATE PENDING
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: ACTIVE
CPU times: user 50 ms, sys: 15.1 ms, total: 65 ms
Wall time: 13min


데이터 세트 가져오기가 활성화되면 SIMS, 개인별 순위, 인기 카운트, HRNN을 사용하여 모델을 구축할 준비가 된 것입니다. 이 프로세스는 다른 노트북에서도 계속됩니다. 다음 셀을 실행하여 다음 노트북에 사용할 몇 가지 값을 저장하세요.

## 솔루션 생성 <a class="anchor" id="solutions"></a>
[맨 위로 이동](#top)

이 노트북에서는 다음 레시피로 솔루션을 만듭니다.

1. HRNN


Amazon Personalize에서는 알고리즘의 특정 변형 버전을 레시피라고 합니다. 레시피마다 적용되는 상황이 다릅니다. 훈련된 모델을 솔루션이라고 하며, 각 솔루션은 모델을 훈련할 때 특정 데이터 볼륨과 관련된 여러 버전을 가질 수 있습니다.

먼저 지원되는 모든 레시피를 나열하겠습니다. 이렇게 하면 모델을 선택하고 모델을 구축하는 데 사용할 수 있습니다.

In [32]:
personalize.list_recipes()

{'recipes': [{'name': 'aws-hrnn',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-hrnn',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2020, 5, 22, 6, 20, 26, 250000, tzinfo=tzlocal())},
  {'name': 'aws-hrnn-coldstart',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-hrnn-coldstart',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2020, 5, 22, 6, 20, 26, 250000, tzinfo=tzlocal())},
  {'name': 'aws-hrnn-metadata',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-hrnn-metadata',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2020, 5, 22, 6, 20, 26, 250000, tzinfo=tzlocal())},
  {'name': 'aws-personalized-ranking',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-personalized-ranking',
   's

### 개인별 순위

개인별 순위는 HRNN의 흥미로운 응용 사례 중 하나입니다. 이 알고리즘은 단순히 사용자에게 가장 유망한 항목을 추천하는 것이 아니라, 사용자는 물론 항목의 목록도 입력으로 받습니다. 그런 다음 사용자에게 가장 적합한 순서대로 항목이 다시 렌더링됩니다. 이 사용 사례는 장르를 필터링하거나 특정 사용자에 대해 보다 효과적으로 정렬하고 싶은 광범위한 컬렉션이 있는 등의 경우에 사용합니다.

이 사용 사례의 경우 LastFM 데이터를 사용합니다. 특정 음반사가 사용자에게 자사 아티스트를 추천하기 위해 돈을 지불하는 시나리오를 떠올려볼 수 있습니다. 즉, 추천하고 싶은 아티스트의 리스트는 알고 있지만, 이 아티스트들 중 각 사용자가 가장 좋아할 아티스트를 찾고자 합니다. 이전 태그 기록을 기준으로 각 사용자의 아티스트 목록을 다시 정렬하기 위해 개인별 순위를 사용합니다.

지난 번처럼 레시피를 선택하는 것부터 시작합니다.

In [33]:
rerank_recipe_arn = "arn:aws:personalize:::recipe/aws-personalized-ranking"

#### 솔루션 생성

이전 솔루션과 마찬가지로 먼저 솔루션을 생성합니다. 이 단계에서 데이터 세트 ARN을 제공하지만 모델은 아직 훈련되지 않았습니다. 따라서 훈련된 모델이 아니라 식별자로 생각해야 합니다.

In [34]:
rerank_create_solution_response = personalize.create_solution(
    name = "personalize-ranking",
    datasetGroupArn = dataset_group_arn,
    recipeArn = rerank_recipe_arn
)

rerank_solution_arn = rerank_create_solution_response['solutionArn']
print(json.dumps(rerank_create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:us-east-1:144386903708:solution/personalize-ranking",
  "ResponseMetadata": {
    "RequestId": "ba5cdcc0-de91-4bf0-a06d-dde91ff1b6ac",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 26 May 2020 21:04:58 GMT",
      "x-amzn-requestid": "ba5cdcc0-de91-4bf0-a06d-dde91ff1b6ac",
      "content-length": "89",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### 솔루션 버전 생성

솔루션을 찾은 후에는 모델 훈련을 완료하기 위해 버전을 만들어야 합니다. 훈련을 완료하는 데 25분 이상 걸릴 수 있으며, 이 예의 데이터 세트를 사용하여 이 레시피를 완료하는 데 평균 35분이 소요됩니다. 일반적으로 작업이 완료될 때까지 while 루프를 사용하여 폴링합니다. 하지만 이 작업은 다른 셀의 실행을 차단하며, 여기서 목표는 많은 모델을 만들고 신속하게 배포하는 것입니다. 따라서 노트북의 아래쪽에 있는 모든 솔루션에 대해 while 루프를 설정합니다. 또한 AWS 콘솔에서 진행 상황을 확인하는 방법에 대한 지침을 찾을 수 있습니다.

In [35]:
rerank_create_solution_version_response = personalize.create_solution_version(
    solutionArn = rerank_solution_arn
)

In [36]:
rerank_solution_version_arn = rerank_create_solution_version_response['solutionVersionArn']
print(json.dumps(rerank_create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:144386903708:solution/personalize-ranking/0f00a819",
  "ResponseMetadata": {
    "RequestId": "b07c347c-48aa-4650-86f6-53197351a62f",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 26 May 2020 21:04:58 GMT",
      "x-amzn-requestid": "b07c347c-48aa-4650-86f6-53197351a62f",
      "content-length": "105",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### 솔루션 생성 상태 보기

예고해 드린 대로, 콘솔에서 상태 업데이트를 보는 방법은 다음과 같습니다.

* 다른 브라우저 탭에서 이 노트북 인스턴스를 열 때 AWS 콘솔이 이미 열려 있어야 합니다.
* 해당 탭으로 전환하고 맨 위에서 `Personalize` 서비스를 검색한 다음 해당 서비스 페이지로 이동합니다.
* `View dataset groups`를 클릭합니다.
* 데이터 세트 그룹의 이름을 클릭합니다. 이름에 POC가 있을 가능성이 높습니다.
* `Solutions and recipes`를 클릭합니다.
* 이제 솔루션 버전 상태가 표시된 열을 포함하여, 위에서 생성한 모든 솔루션 목록이 표시됩니다. `Active` 상태가 되면 솔루션을 검토할 준비가 된 것입니다. 또한 배포할 수도 있습니다.

또는 아래의 셀을 실행하여 솔루션 버전 생성 상태를 추적합니다.

In [38]:
in_progress_solution_versions = [
    rerank_solution_version_arn
]

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    for solution_version_arn in in_progress_solution_versions:
        version_response = personalize.describe_solution_version(
            solutionVersionArn = solution_version_arn
        )
        status = version_response["solutionVersion"]["status"]
        
        if status == "ACTIVE":
            print("Build succeeded for {}".format(solution_version_arn))
            in_progress_solution_versions.remove(solution_version_arn)
        elif status == "CREATE FAILED":
            print("Build failed for {}".format(solution_version_arn))
            in_progress_solution_versions.remove(solution_version_arn)
    
    if len(in_progress_solution_versions) <= 0:
        break
    else:
        print("At least one solution build is still in progress")
        
    time.sleep(60)

Build succeeded for arn:aws:personalize:us-east-1:144386903708:solution/personalize-ranking/0f00a819


## 캠페인 생성 <a class="anchor" id="create"></a>
[맨 위로 이동](#top)

캠페인은 호스팅된 솔루션 버전으로, 추천을 쿼리할 수 있는 엔드포인트입니다. 요금은 처리 용량(초당 사용자의 개인화 요청)을 추정하여 정해집니다. 캠페인을 배포할 때 최소 초당 처리량(TPS) 값을 설정합니다. 이 서비스는 AWS 내의 다른 서비스와 마찬가지로 수요에 따라 자동으로 확장되지만, 대기 시간이 중요한 경우에는 더 많은 수요에 대비해 미리 프로비저닝하는 것이 좋습니다. 이 POC 및 데모에서는 모든 최소 처리량 임계값을 1로 설정합니다. 자세한 내용은 [요금 페이지](https://aws.amazon.com/personalize/pricing/)를 참조하세요.

이제 캠페인을 구축해 보겠습니다.

### 개인별 순위

개인화된 순위 솔루션 버전에 대한 캠페인을 배포합니다. 캠페인을 배포하는 데 10분 정도 걸릴 수 있습니다. 일반적으로 작업이 완료될 때까지 while 루프를 사용하여 폴링합니다. 하지만 이 작업은 다른 셀의 실행을 차단하며, 여기서 목표는 여러 캠페인을 만드는 것입니다. 따라서 노트북의 아래쪽에 있는 모든 캠페인에 대해 while 루프를 설정합니다. 또한 AWS 콘솔에서 진행 상황을 확인하는 방법에 대한 지침을 찾을 수 있습니다.

In [39]:
rerank_create_campaign_response = personalize.create_campaign(
    name = "personalize-poc-rerank",
    solutionVersionArn = rerank_solution_version_arn,
    minProvisionedTPS = 1
)

rerank_campaign_arn = rerank_create_campaign_response['campaignArn']
print(json.dumps(rerank_create_campaign_response, indent=2))

{
  "campaignArn": "arn:aws:personalize:us-east-1:144386903708:campaign/personalize-poc-rerank",
  "ResponseMetadata": {
    "RequestId": "61fd83dd-e409-4433-a2b9-667aecdecb0a",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 26 May 2020 21:48:55 GMT",
      "x-amzn-requestid": "61fd83dd-e409-4433-a2b9-667aecdecb0a",
      "content-length": "92",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### 캠페인 생성 상태 보기

예고해 드린 대로, 콘솔에서 상태 업데이트를 보는 방법은 다음과 같습니다.

* 다른 브라우저 탭에서 이 노트북 인스턴스를 열 때 AWS 콘솔이 이미 열려 있어야 합니다.
* 해당 탭으로 전환하고 맨 위에서 `Personalize` 서비스를 검색한 다음 해당 서비스 페이지로 이동합니다.
* `View dataset groups`를 클릭합니다.
* 데이터 세트 그룹의 이름을 클릭합니다. 이름에 POC가 있을 가능성이 높습니다.
* `Campaigns`를 클릭합니다.
* 이제 캠페인 상태가 표시된 열을 포함하여, 위에서 생성한 모든 캠페인의 목록이 표시됩니다. `Active` 상태가 되면 캠페인을 쿼리할 수 있습니다.

또는 아래의 셀을 실행하여 캠페인 생성 상태를 추적합니다.

In [41]:
in_progress_campaigns = [
    rerank_campaign_arn
]

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    for campaign_arn in in_progress_campaigns:
        version_response = personalize.describe_campaign(
            campaignArn = campaign_arn
        )
        status = version_response["campaign"]["status"]
        
        if status == "ACTIVE":
            print("Build succeeded for {}".format(campaign_arn))
            in_progress_campaigns.remove(campaign_arn)
        elif status == "CREATE FAILED":
            print("Build failed for {}".format(campaign_arn))
            in_progress_campaigns.remove(campaign_arn)
    
    if len(in_progress_campaigns) <= 0:
        break
    else:
        print("At least one campaign build is still in progress")
        
    time.sleep(60)

At least one campaign build is still in progress
At least one campaign build is still in progress
At least one campaign build is still in progress
At least one campaign build is still in progress
At least one campaign build is still in progress
At least one campaign build is still in progress
At least one campaign build is still in progress
At least one campaign build is still in progress
Build succeeded for arn:aws:personalize:us-east-1:144386903708:campaign/personalize-poc-rerank


## 캠페인과 상호 작용 <a class="anchor" id="interact"></a>
[맨 위로 이동](#top)

이제 모든 캠페인이 배포되고 활성화되었으므로 API 호출을 통해 추천을 받을 수 있습니다. 각 캠페인은 약간 다른 방식으로 작동하는 다른 레시피를 기반으로, 서로 다른 사용 사례를 지원합니다. 각 캠페인은 이전 노트북과 다른 순서로 진행되며, 복잡성이 발생할 수 있는 경우 오름차순(즉, 가장 간단한 것부터)에 따라 처리됩니다.

먼저 Personalize 캠페인이 반환한 결과를 이해할 수 있도록 지원 함수를 만들어 보겠습니다. Personalize는 `item_id`만 반환합니다. 이 함수는 데이터를 압축하기에 좋지만, 사용자가 읽을 수 있는 노트북 결과를 얻으려면 데이터베이스나 조회 테이블을 쿼리해야 합니다. 헬퍼 함수를 생성하여 FM 데이터 세트에서 사용자가 읽을 수 있는 결과를 반환하겠습니다.

먼저 조회 테이블에 사용할 수 있는 데이터 세트를 로드합니다.

In [42]:
# Create a dataframe for the items by reading in the correct source CSV
items_df = pd.read_csv(data_dir + '/artists.dat', delimiter='\t', index_col=0)

# Render some sample data
items_df.head(5)

,name,url,pictureURL
id,,,
1,MALICE MIZER,http://www.last.fm/music/MALICE+MIZER,http://userserve-ak.last.fm/serve/252/10808.jpg
2,Diary of Dreams,http://www.last.fm/music/Diary+of+Dreams,http://userserve-ak.last.fm/serve/252/3052066.jpg
3,Carpathian Forest,http://www.last.fm/music/Carpathian+Forest,http://userserve-ak.last.fm/serve/252/40222717...
4,Moi dix Mois,http://www.last.fm/music/Moi+dix+Mois,http://userserve-ak.last.fm/serve/252/54697835...
5,Bella Morte,http://www.last.fm/music/Bella+Morte,http://userserve-ak.last.fm/serve/252/14789013...


ID 열을 인덱스 열로 정의하면 ID를 쿼리하는 것만으로 아티스트를 반환할 수 있습니다.

In [43]:
item_id_example = 987
artist = items_df.loc[item_id_example]['name']
print(artist)

Earth, Wind & Fire


그렇게 나쁘지는 않지만, 코드의 모든 곳에서 이 작업을 반복하면 더 복잡해집니다. 그래서 아래의 함수가 이를 정리해줄 것입니다.

In [44]:
def get_artist_by_id(artist_id, artist_df=items_df):
    """
    This takes in an artist_id from Personalize so it will be a string,
    converts it to an int, and then does a lookup in a default or specified
    dataframe.
    
    A really broad try/except clause was added in case anything goes wrong.
    
    Feel free to add more debugging or filtering here to improve results if
    you hit an error.
    """
    try:
        return artist_df.loc[int(artist_id)]['name']
    except:
        return "Error obtaining artist"

이제 오류 캐칭 기능을 확인하기 위해 몇 가지 간단한 값을 테스트해 보겠습니다.

In [45]:
# A known good id
print(get_artist_by_id(artist_id="987"))
# A bad type of value
print(get_artist_by_id(artist_id="987.9393939"))
# Really bad values
print(get_artist_by_id(artist_id="Steve"))

Earth, Wind & Fire
Error obtaining artist
Error obtaining artist


좋습니다. 이제 결과를 렌더링할 수 있습니다. 

In [47]:
users_df = pd.read_csv(data_dir + '/user_artists.dat', delimiter='\t', index_col=0)
# Render some sample data
users_df.head(5)

,artistID,weight
userID,,
2,51,13883
2,52,11690
2,53,11351
2,54,10300
2,55,8983


### 개인별 순위

개인화된 순위의 핵심 사용 사례는 항목 컬렉션을 가지고, 사용자에게 우선 순위 또는 가능한 관심도 순서로 렌더링하는 것입니다. 이를 보여주려면 임의의 사용자와 25개 항목의 무작위 컬렉션이 필요합니다.

In [48]:
rerank_user = users_df.sample(1).index.tolist()[0]
rerank_items = items_df.sample(25).index.tolist()

이제 입력 데이터를 보여주는 멋진 데이터 프레임을 구축합니다.

In [49]:
rerank_list = []
for item in rerank_items:
    artist = get_artist_by_id(item)
    rerank_list.append(artist)
rerank_df = pd.DataFrame(rerank_list, columns = [rerank_user])
rerank_df

,1303
0,Priestess
1,Пелагея
2,Cosmic Gate
3,Mujuice
4,Prism
5,WIZO
6,Love of Lesbian
7,Mayhem
8,Taking Dawn
9,Universum


그런 다음 개인화된 순위 API를 호출합니다.

In [50]:
# Convert user to string:
user_id = str(rerank_user)
rerank_item_list = []
for item in rerank_items:
    rerank_item_list.append(str(item))
    
# Get recommended reranking
get_recommendations_response_rerank = personalize_runtime.get_personalized_ranking(
        campaignArn = rerank_campaign_arn,
        userId = user_id,
        inputList = rerank_item_list
)

get_recommendations_response_rerank

{'ResponseMetadata': {'RequestId': 'fd2ea40f-11f5-461c-b678-6a6a08bdb416',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/json',
   'date': 'Tue, 26 May 2020 22:02:55 GMT',
   'x-amzn-requestid': 'fd2ea40f-11f5-461c-b678-6a6a08bdb416',
   'content-length': '1442',
   'connection': 'keep-alive'},
  'RetryAttempts': 0},
 'personalizedRanking': [{'itemId': '1047', 'score': 0.1730128},
  {'itemId': '1046', 'score': 0.1639514},
  {'itemId': '1254', 'score': 0.1101557},
  {'itemId': '18692', 'score': 0.0762716},
  {'itemId': '5783', 'score': 0.0678702},
  {'itemId': '13434', 'score': 0.067667},
  {'itemId': '8915', 'score': 0.0616803},
  {'itemId': '1305', 'score': 0.0322442},
  {'itemId': '8543', 'score': 0.0297265},
  {'itemId': '11979', 'score': 0.0286533},
  {'itemId': '6820', 'score': 0.025196},
  {'itemId': '4154', 'score': 0.0246835},
  {'itemId': '2312', 'score': 0.0228279},
  {'itemId': '2158', 'score': 0.0179717},
  {'itemId': '2036', 'score': 0.0176177},
 

나란히 비교할 수 있도록, 순위가 다시 매겨진 항목을 원래 데이터 프레임에 두 번째 열로 추가합니다.

In [51]:
ranked_list = []
item_list = get_recommendations_response_rerank['personalizedRanking']
for item in item_list:
    artist = get_artist_by_id(item['itemId'])
    ranked_list.append(artist)
ranked_df = pd.DataFrame(ranked_list, columns = ['Re-Ranked'])
rerank_df = pd.concat([rerank_df, ranked_df], axis=1)
rerank_df

,1303,Re-Ranked
0,Priestess,McFly
1,Пелагея,New Found Glory
2,Cosmic Gate,Mayhem
3,Mujuice,Farid Farjad
4,Prism,Priestess
5,WIZO,Пелагея
6,Love of Lesbian,How to Destroy Angels
7,Mayhem,Cosmic Gate
8,Taking Dawn,Prism
9,Universum,Murder by Death


위에서 모델의 사용자에 대한 이해를 바탕으로 각 항목이 어떻게 다시 정렬되었는지 확인할 수 있습니다. 이 작업은 프로모션 목록과 같이 사용자에게 노출할 항목 컬렉션이 있거나 범주를 필터링하여 가장 적합한 항목을 표시하려는 경우에 널리 사용됩니다.

## 캠페인 정리

맨 먼저 캠페인을 정리합니다. 캠페인의 기반이 되는 솔루션 버전을 삭제하려면 먼저 캠페인을 삭제해야 합니다.

아래 코드는 계정에 있는 모든 캠페인을 나열합니다.

In [ ]:
paginator = personalize.get_paginator('list_campaigns')
for paginate_result in paginator.paginate():
    for campaign in paginate_result["campaigns"]:
        print(campaign["campaignArn"])

ARN 목록을 확인하여 삭제할 캠페인을 결정합니다. 그런 다음 아래 코드를 사용하여 ARN을 삽입하는 방식으로 캠페인을 삭제합니다.

In [ ]:
personalize.delete_campaign(
    campaignArn = "INSERT ARN HERE"
)

## 솔루션 정리

다음으로 솔루션을 정리합니다. 아래 코드는 계정에 있는 모든 솔루션을 나열합니다.

In [ ]:
paginator = personalize.get_paginator('list_solutions')
for paginate_result in paginator.paginate():
    for solution in paginate_result["solutions"]:
        print(solution["solutionArn"])

ARN 목록을 확인하여 삭제할 솔루션을 결정합니다. 그런 다음 아래 코드를 사용하여 ARN을 삽입하는 방식으로 삭제합니다.

In [ ]:
personalize.delete_solution(
    solutionArn = "INSERT ARN HERE"
)

## 데이터 세트 정리

다음으로 데이터 세트를 정리합니다. 아래 코드는 계정에 있는 모든 데이터 세트를 나열합니다.

In [ ]:
paginator = personalize.get_paginator('list_datasets')
for paginate_result in paginator.paginate():
    for datasets in paginate_result["datasets"]:
        print(datasets["datasetArn"])

ARN 목록을 확인하여 삭제할 데이터 세트를 결정합니다. 그런 다음 아래 코드를 사용하여 ARN을 삽입하는 방식으로 삭제합니다.

In [ ]:
personalize.delete_dataset(
    datasetArn = "INSERT ARN HERE"
)

## 스키마 정리

다음으로 스키마를 정리합니다. 아래 코드는 계정에 있는 모든 스키마를 나열합니다.

In [ ]:
paginator = personalize.get_paginator('list_schemas')
for paginate_result in paginator.paginate():
    for schema in paginate_result["schemas"]:
        print(schema["schemaArn"])

ARN 목록을 확인하여 삭제할 스키마를 결정합니다. 그런 다음 아래 코드를 사용하여 ARN을 삽입하는 방식으로 삭제합니다.

In [ ]:
personalize.delete_schema(
    schemaArn = "INSERT ARN HERE"
)

## 데이터 세트 그룹 정리

마지막으로, 데이터 세트 그룹을 정리합니다. 아래 코드는 계정에 있는 모든 데이터 세트 그룹을 나열합니다.

In [ ]:
paginator = personalize.get_paginator('list_dataset_groups')
for paginate_result in paginator.paginate():
    for dataset_group in paginate_result["datasetGroups"]:
        print(dataset_group["datasetGroupArn"])

ARN 목록을 확인하여 삭제할 데이터 세트 그룹을 결정합니다. 그런 다음 아래 코드를 사용하여 ARN을 삽입하는 방식으로 삭제합니다.

In [ ]:
personalize.delete_dataset_group(
    datasetGroupArn = "INSERT ARN HERE"
)

## S3 버킷 및 IAM 역할 정리

원하는 경우 IAM 역할과 워크숍을 진행하는 동안 생성한 S3 버킷을 삭제할 수 있습니다.

아래 코드를 사용하여 계정의 모든 IAM 역할을 나열하는 것부터 시작합니다.

In [ ]:
iam = boto3.client('iam')

paginator = iam.get_paginator('list_roles')
for paginate_result in paginator.paginate():
    for roles in paginate_result["Roles"]:
        print(roles["RoleName"])

삭제할 역할의 이름을 식별합니다.

아직 정책이 연결되어 있는 IAM 역할은 삭제할 수 없습니다. 따라서 관련 역할을 식별한 후, 해당 역할에 연결된 정책을 나열해 봅니다.

In [ ]:
iam.list_attached_role_policies(
    RoleName = "INSERT ROLE NAME HERE"
)

아래 코드를 사용하여 위의 결과에서 정책을 분리해야 합니다. 연결된 정책마다 이 작업을 반복합니다.

In [ ]:
iam.detach_role_policy(
    RoleName = "INSERT ROLE NAME HERE",
    PolicyArn = "INSERT ARN HERE"
)

이제 IAM 역할을 삭제할 수 있습니다.

In [ ]:
iam.delete_role(
    RoleName = "INSERT ROLE NAME HERE"
)

S3 버킷을 삭제하려면 먼저 버킷이 비어 있어야 합니다. S3 버킷을 삭제하는 가장 쉬운 방법은 AWS 콘솔에서 S3으로 이동하여 버킷의 객체를 삭제한 다음 S3 버킷 자체를 삭제하는 것입니다.